In [1]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

Working on a  Windows 10
Python version  3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
Pandas version  1.5.3
bifacial_radiance version  0.4.4.dev25+g9486645.d20240925


# 21 - Weather to Module Performance
## Modeling Performance, an End to End Simulation

This tutorial shows how to use the new function on bifacial_radiance calculatePerformanceModule performance, as well as how to find CEC Module parameters.


In [2]:
import os
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_21')

if not os.path.exists(testfolder): os.mkdir(testfolder)

# Another option using relative address; for some operative systems you might need '/' instead of '\'
# testfolder = os.path.abspath(r'..\..\bifacial_radiance\TEMP')  

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21


In [3]:
import bifacial_radiance
import numpy as np
import pandas as pd
import pvlib 

bifacial_radiance.__version__

'0.4.4.dev25+g9486645.d20240925'

## Geting a CEC Module to pass into demo.makeModule

In [4]:
url = 'https://raw.githubusercontent.com/NREL/SAM/patch/deploy/libraries/CEC%20Modules.csv'
db = pd.read_csv(url, index_col=0) # Reading this might take 1 min or so, the database is big.

Find the module that you want. In this case we know it's a SunPower of model SPR-E19-310-COM. 

Make sure you select only 1 module from the database -- sometimes there are similar names.

In [5]:
modfilter2 = db.index.str.startswith('SunPower') & db.index.str.endswith('SPR-E19-310-COM')
print(modfilter2)
CECMod = db[modfilter2]
print(len(CECMod), " modules selected. Name of 1st entry: ", CECMod.index[0])

[False False False ... False False False]
1  modules selected. Name of 1st entry:  SunPower SPR-E19-310-COM


In [6]:
# Selecting only two times as examples
starttime = '01_13_11';  endtime = '01_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
demo.setGround(0.2)

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.


The CEC data should be passed into the ModuleObj, either at time of creation, or sometime before it is passed into makeScene.

In [7]:
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 


Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



The same data could instead be passed after the ModuleObj's definition, or at time of performance analysis:

In [8]:
mymodule.addCEC(CECMod)

In [9]:
# Let's make a second module, and set it to the default Prism Solar module type
mymodule2 = demo.makeModule(name='test', x=1, y=2, bifi=0.8, CECMod=None) 


Module Name: test
Module test updated in module.json
Pre-existing .rad file objects\test.rad will be overwritten



We're going to set up two scenes, each with a different module type!

In [10]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata = metdata, cumulativesky = False)
trackerdict = demo.gendaylit1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)

Creating ~2 skyfiles. 
Created 2 skyfiles in /skies/

Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


Make a second scene with the other module type

In [11]:
sceneDict2 = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':2.5, 'nMods':2, 'nRows': 1, 'originx': -15}
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule2, sceneDict=sceneDict2, append=True)



Making ~2 .rad files for gendaylit 1-axis workflow (this takes a minute..)
2 Radfiles created in /objects/


In [12]:
# Compile both scenes into one octfile.  Run 2 different analyses, one on each scene with different front and rear y scan
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(sensorsy=[1,3], append=False)
trackerdict = demo.analysis1axis(sensorsy=[3,2], sceneNum=1)


Making 2 octfiles in root directory.
Created 1axis_2021-01-13_1100.oct
Created 1axis_2021-01-13_1200.oct

Analysis1axis: append=False. Over-writing any existing `AnalysisObj` in trackerdict.

Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1100_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-01-13_1100_Scene0_Row1_Module3_Back.csv
Index: 2021-01-13_1100. Wm2Front: 254.26609999999997. Wm2Back: 39.97652333333334
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Front
Linescan in process: 1axis_2021-01-13_1200_Scene0_Row1_Module3_Back
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module3_Front.csv
Saved: results\irr_1axis_2021-01-13_1200_Scene0_Row1_Module3_Back.csv
Index: 2021-01-13_1200. Wm2Front: 253.4217. Wm2Back: 38.20968333333333
Linescan in process: 1axis_2021-01-13_1100_Scene1_Row1_Module1_Front
Linescan in process: 1axis_202

In [13]:
# Include an AgriPV groundscan too
trackerdict = demo.analysis1axisground(sceneNum=1, sensorsground=10, customname='Silvanas_')

Linescan in process: 1axis_groundscan_2021-01-13_1100Silvanas__Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1100Silvanas__Row1_Module1.csv
Index: 2021-01-13_1100. Wm2Ground: 204.79288999999997. sensorsground: 10
Linescan in process: 1axis_groundscan_2021-01-13_1200Silvanas__Row1_Module1_Front
Saved: results\irr_1axis_groundscan_2021-01-13_1200Silvanas__Row1_Module1.csv
Index: 2021-01-13_1200. Wm2Ground: 237.48879000000002. sensorsground: 10


In [14]:
# show the initial irradiance results before continuing:
demo.results

,timestamp,name,modNum,rowNum,sceneNum,Wm2Front,Wm2Back,backRatio,x,y,z,mattype,rearX,rearY,rearZ,rearMat,surf_azm,surf_tilt,theta
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,[254.26609999999997],"[39.59574, 39.60611, 40.72772]",0.157223,[0.01462555],[0.0],[1.514648],[a2.0.a0.test-module.6457],"[0.3581543, -0.0007324219, -0.3596191]","[0.0, 0.0, 0.0]","[1.151367, 1.499512, 1.847656]","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",90.0,44.14,-44.14
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[253.0651, 253.8192, 253.7649]","[0.0, 0.0]",0.0,"[-14.625, -14.98389, -15.34277]","[0.0, 0.0, 0.0]","[2.166016, 2.51416, 2.862305]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[-14.75781, -15.23633]","[0.0, 0.0]","[2.267578, 2.731934]","[a0.0.a0.test.6457, a0.0.a0.test.6457]",90.0,44.14,-44.14
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100Silvanas_,1,1,1,"[199.1224, 201.452, 204.5499, 206.0879, 207.14...",NaN,NaN,"[-15.0, -14.44434, -13.88867, -13.33301, -12.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04998779, 0.04998779, 0.04998779, 0.0499877...","[groundplane, groundplane, groundplane, ground...",NaN,NaN,NaN,NaN,90.0,44.14,-44.14
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,[253.4217],"[37.89736, 37.81097, 38.92072]",0.150775,[0.007595062],[0.0],[1.519531],[a2.0.a0.test-module.6457],"[0.4658203, -0.0002441406, -0.4663086]","[0.0, 0.0, 0.0]","[1.318359, 1.499146, 1.679932]","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",90.0,21.20,-21.20
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[252.4514, 252.5719, 252.6928]","[0.0, 0.0]",0.0,"[-14.52344, -14.9895, -15.45557]","[0.0, 0.0, 0.0]","[2.337891, 2.518677, 2.699463]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[-14.6875, -15.30908]","[0.0, 0.0]","[2.378906, 2.619995]","[a0.0.a0.test.6457, a0.0.a0.test.6457]",90.0,21.20,-21.20
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200Silvanas_,1,1,1,"[227.9177, 230.1437, 233.8727, 236.2223, 238.9...",NaN,NaN,"[-15.0, -14.44434, -13.88867, -13.33301, -12.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04998779, 0.04998779, 0.04998779, 0.0499877...","[groundplane, groundplane, groundplane, ground...",NaN,NaN,NaN,NaN,90.0,21.20,-21.20


## Calculating the Performance and Exporting the Results to a CSV

In [15]:
# Calculate performance.
pd.set_option('display.max_columns', 1000); 
compiledResults = demo.calculatePerformance1axis()
print(f'\nCompiled results:\n')
display(compiledResults)


No CECModule data passed; using default for Prism Solar BHC72-400

Compiled results:



,timestamp,name,modNum,rowNum,sceneNum,Wm2Front,Wm2Back,backRatio,x,y,z,mattype,rearX,rearY,rearZ,rearMat,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Pout_Gfront,BGG,BGE,Mismatch,Pout,Wind Speed,DNI,DHI,GHI,module_CEC_name
0,2021-01-13_1100,1axis_2021-01-13_1100_Scene0,3,1,0,[254.26609999999997],"[39.59574, 39.60611, 40.72772]",0.157223,[0.01462555],[0.0],[1.514648],[a2.0.a0.test-module.6457],"[0.3581543, -0.0007324219, -0.3596191]","[0.0, 0.0, 0.0]","[1.151367, 1.499512, 1.847656]","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",290.244971,254.266100,39.976523,91.775968,80.310389,14.150086,14.276583,0.022931,91.754923,3.8,144,149,211,SunPower SPR-E19-310-COM
1,2021-01-13_1100,1axis_2021-01-13_1100_Scene1,1,1,1,"[253.0651, 253.8192, 253.7649]","[0.0, 0.0]",0.0,"[-14.625, -14.98389, -15.34277]","[0.0, 0.0, 0.0]","[2.166016, 2.51416, 2.862305]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[-14.75781, -15.23633]","[0.0, 0.0]","[2.267578, 2.731934]","[a0.0.a0.test.6457, a0.0.a0.test.6457]",253.549733,253.549733,0.000000,106.072139,106.072139,0.000000,0.000000,0.000000,106.072139,3.8,144,149,211,Prism Solar Technologies Inc. BHC72-400
2,2021-01-13_1100,1axis_groundscan_2021-01-13_1100Silvanas_,1,1,1,"[199.1224, 201.452, 204.5499, 206.0879, 207.14...",NaN,NaN,"[-15.0, -14.44434, -13.88867, -13.33301, -12.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04998779, 0.04998779, 0.04998779, 0.0499877...","[groundplane, groundplane, groundplane, ground...",NaN,NaN,NaN,NaN,204.792890,204.792890,0.000000,85.584512,85.584512,0.000000,0.000000,0.000000,85.584512,3.8,144,149,211,Prism Solar Technologies Inc. BHC72-400
3,2021-01-13_1200,1axis_2021-01-13_1200_Scene0,3,1,0,[253.4217],"[37.89736, 37.81097, 38.92072]",0.150775,[0.007595062],[0.0],[1.519531],[a2.0.a0.test-module.6457],"[0.4658203, -0.0002441406, -0.4663086]","[0.0, 0.0, 0.0]","[1.318359, 1.499146, 1.679932]","[a2.0.a0.test-module.2310, a2.0.a0.test-module...",287.810415,253.421700,38.209683,90.333308,79.448352,13.569759,13.700669,0.022662,90.312836,3.9,97,200,249,SunPower SPR-E19-310-COM
4,2021-01-13_1200,1axis_2021-01-13_1200_Scene1,1,1,1,"[252.4514, 252.5719, 252.6928]","[0.0, 0.0]",0.0,"[-14.52344, -14.9895, -15.45557]","[0.0, 0.0, 0.0]","[2.337891, 2.518677, 2.699463]","[a0.0.a0.test.6457, a0.0.a0.test.6457, a0.0.a0...","[-14.6875, -15.30908]","[0.0, 0.0]","[2.378906, 2.619995]","[a0.0.a0.test.6457, a0.0.a0.test.6457]",252.572033,252.572033,0.000000,104.950213,104.950213,0.000000,0.000000,0.000000,104.950213,3.9,97,200,249,Prism Solar Technologies Inc. BHC72-400
5,2021-01-13_1200,1axis_groundscan_2021-01-13_1200Silvanas_,1,1,1,"[227.9177, 230.1437, 233.8727, 236.2223, 238.9...",NaN,NaN,"[-15.0, -14.44434, -13.88867, -13.33301, -12.7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.04998779, 0.04998779, 0.04998779, 0.0499877...","[groundplane, groundplane, groundplane, ground...",NaN,NaN,NaN,NaN,237.488790,237.488790,0.000000,98.665486,98.665486,0.000000,0.000000,0.000000,98.665486,3.9,97,200,249,Prism Solar Technologies Inc. BHC72-400


In [16]:
demo.exportTrackerDict(savefile=os.path.join('results','Final_Results.csv'),reindex=False)
pd.read_csv(os.path.join('results','Final_Results.csv'))

Exporting TrackerDict
3
3
1
1


,Unnamed: 0,timestamp,rowNum,modNum,sceneNum,name,Wm2Front,Wm2Back,DNI,DHI,GHI,temp_air,wind_speed,theta,surf_tilt,surf_azm,POA_eff,Gfront_mean,Grear_mean,Pout_raw,Mismatch,Pout,Pout_Gfront
0,0,2021-01-13_1100,1,3,0,1axis_2021-01-13_1100_Scene0,[254.2661],[39.59574 39.60611 40.72772],144,149,211,NaN,3.8,-44.14,44.14,90.0,290.244971,254.266100,39.976523,91.775968,0.022931,91.754923,80.310389
1,1,2021-01-13_1100,1,1,1,1axis_2021-01-13_1100_Scene1,[253.0651 253.8192 253.7649],[0. 0.],144,149,211,NaN,3.8,-44.14,44.14,90.0,253.549733,253.549733,0.000000,106.072139,0.000000,106.072139,106.072139
2,2,2021-01-13_1100,1,1,1,1axis_groundscan_2021-01-13_1100Silvanas_,[199.1224 201.452 204.5499 206.0879 207.1438 ...,NaN,144,149,211,NaN,3.8,-44.14,44.14,90.0,204.792890,204.792890,0.000000,85.584512,0.000000,85.584512,85.584512
3,3,2021-01-13_1200,1,3,0,1axis_2021-01-13_1200_Scene0,[253.4217],[37.89736 37.81097 38.92072],97,200,249,NaN,3.9,-21.20,21.20,90.0,287.810415,253.421700,38.209683,90.333308,0.022662,90.312836,79.448352
4,4,2021-01-13_1200,1,1,1,1axis_2021-01-13_1200_Scene1,[252.4514 252.5719 252.6928],[0. 0.],97,200,249,NaN,3.9,-21.20,21.20,90.0,252.572033,252.572033,0.000000,104.950213,0.000000,104.950213,104.950213
5,5,2021-01-13_1200,1,1,1,1axis_groundscan_2021-01-13_1200Silvanas_,[227.9177 230.1437 233.8727 236.2223 238.9651 ...,NaN,97,200,249,NaN,3.9,-21.20,21.20,90.0,237.488790,237.488790,0.000000,98.665486,0.000000,98.665486,98.665486


## Now look at gencumulativesky tracking workflow

In [17]:
starttime = '01_13_11';  endtime = '12_13_12'
demo = bifacial_radiance.RadianceObj('tutorial_21', path = testfolder) # Create a RadianceObj 'object'
weatherfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(weatherFile=weatherfile, starttime=starttime, endtime=endtime)
#metdata = demo.readWeatherFile(weatherFile=weatherfile)
demo.setGround(0.2)
mymodule = demo.makeModule(name='test-module', x=1, y=2, bifi=0.9, CECMod=CECMod) 

path = C:\Users\cdeline\Documents\Python Scripts\Bifacial_Radiance\bifacial_radiance\TEMP\Tutorial_21
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Filtering dates
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos
Loading albedo, 1 value(s), 0.200 avg
1 nonzero albedo values.

Module Name: test-module
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



In [18]:
sceneDict = {'tilt': 0, 'azimuth': 180, 'pitch': 5,'hub_height':1.5, 'nMods':5, 'nRows': 2}
trackerdict = demo.set1axis(metdata=metdata, cumulativesky=True, limit_angle=15, backtrack=False)
trackerdict = demo.genCumSky1axis()
trackerdict = demo.makeScene1axis(trackerdict, module = mymodule, sceneDict = sceneDict)
trackerdict = demo.makeOct1axis()
trackerdict = demo.analysis1axis(modWanted = [2,4], sensorsy=[2,3])
trackerdict = demo.analysis1axisground(sensorsground=10)

Saving file EPWs\1axis_-15.0.csv, # points: 1875
Saving file EPWs\1axis_-10.0.csv, # points: 241
Saving file EPWs\1axis_0.0.csv, # points: 2
Saving file EPWs\1axis_5.0.csv, # points: 242
Saving file EPWs\1axis_10.0.csv, # points: 33
Saving file EPWs\1axis_15.0.csv, # points: 1683
message: There were 1833 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-15.0.rad
message: There were 238 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_-10.0.rad
message: There were 2 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_0.0.rad
message: There were 239 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_5.0.rad
message: There were 33 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_10.0.rad
message: There were 1661 sun up hours in this climate file
Total Ibh/Lbh: 0.000000
Created skyfile skies\1axis_15.0.rad

Makin

In [19]:
results = demo.calculatePerformance1axis() # saves to demo.compiledResults and results/Cumulative_Results.csv
print('\nCompiled results:\n')
display(demo.compiledResults)


Compiled results:



,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.360663,1.655209e+06,208936.319433,"[1845370.4100300001, 1840857.22746, 1843526.64...","[1658321.0965, 1652096.3821999999, nan, nan, n...","[211290.74520000003, 206276.09790000002, 20924..."
1,1,3,0,NaN,1.125122e+06,NaN,"[0.0, 0.0, 0.0]","[632528.9293000001, 704530.8141, 871543.067099...","[nan, nan, nan]"
2,1,4,0,10.280712,1.671282e+06,190910.812600,"[1845953.2714499997, 1839710.38887, 1843642.7352]","[1671845.97, 1670718.8309999998, nan, nan, nan...","[194078.7455, 187142.2093, 191511.48300000004]"


In [20]:
# Results are also automatically saved in \results\Cumulative_Results.csv
pd.read_csv(os.path.join('results','Cumulative_Results.csv'))

,rowNum,modNum,sceneNum,BGG,Gfront_mean,Grear_mean,POA_eff,Wm2Front,Wm2Back
0,1,2,0,11.361,1655208.739,208936.319,"[1845370.4100300001, 1840857.22746, 1843526.64...","[1658321.0965, 1652096.3821999999, nan, nan, n...","[211290.74520000003, 206276.09790000002, 20924..."
1,1,3,0,NaN,1125121.777,NaN,"[0.0, 0.0, 0.0]","[632528.9293000001, 704530.8141, 871543.067099...","[nan, nan, nan]"
2,1,4,0,10.281,1671282.400,190910.813,"[1845953.2714499997, 1839710.38887, 1843642.7352]","[1671845.97, 1670718.8309999998, nan, nan, nan...","[194078.7455, 187142.2093, 191511.48300000004]"
